In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from deel.lip.activations import GroupSort2
from deel.lip.layers import (
    FrobeniusDense,
    ScaledL2NormPooling2D,
    SpectralConv2D,
    SpectralDense,
)
from deel.lip.losses import MulticlassHKR, MulticlassKR
from deel.lip.model import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
layers = [
    SpectralDense(100, kernel_initializer="orthogonal"),
    GroupSort2(),
    SpectralDense(100, kernel_initializer="orthogonal"),
    GroupSort2(),
    SpectralDense(32, kernel_initializer="orthogonal"),
    GroupSort2(),
    FrobeniusDense(10, activation=None, use_bias=False, kernel_initializer="orthogonal"),
]

model = Sequential(
    layers,
    k_coef_lip=1.0,
    name="hkr_model",
)

In [ ]:
model.compile(
    # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
    # note also in the case of lipschitz networks, more robustness require more parameters.
    loss=MulticlassHKR(alpha=50, min_margin=0.05),
    optimizer=Adam(1e-3),
    metrics=["accuracy", MulticlassKR()],
)


# load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# standardize and reshape the data
x_train = np.expand_dims(x_train, -1)
mean = x_train.mean()
std = x_train.std()
x_train = (x_train - mean) / std
x_test = np.expand_dims(x_test, -1)
x_test = (x_test - mean) / std
# one hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

In [ ]:
# fit the model
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_test, y_test),
    shuffle=True,
    verbose=0,
)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
# once training is finished you can convert
# SpectralDense layers into Dense layers and SpectralConv2D into Conv2D
# which optimize performance for inference
vanilla_model = model.vanilla_export()

In [ ]:
vanilla_model.summary()

In [ ]:
from decomon import get_adv_box
from decomon.models import clone

In [ ]:
C = Input((10, 10))
convex_domain = {"name": "ball", "p": 2, "eps": tf.Variable(0.0)}

In [ ]:
decomon_model_adv = clone(vanilla_model, back_bounds=[C])

In [ ]:
x_min = x_test[:1] - eps_ / 1000
x_max = x_test[:1] + eps_ / 1000

In [ ]:
box = np.concatenate([x_min[:, None], x_max[:, None]], 1)

In [ ]:
upper = decomon_model_adv.predict([box, C_i])

In [ ]:
x_test

In [ ]:
upper

retrive one radis bound

In [ ]:
vanilla_model.predict(x_test[:1]).argmax()

In [ ]:
y_test[:1].argmax()

In [ ]:
tmp = vanilla_model.predict(x_test[:1])[0]
tmp.sort()
eps_ = (tmp[-1] - tmp[-2]) / np.sqrt(2)

In [ ]:
K.set_value(convex_domain["eps"], eps_)

In [ ]:
C_ = np.diag([1.0] * 10)[None] - y_test[:, :, None]
C_i = C_[:1]

In [ ]:
C_i.shape

In [ ]:
decomon_model_adv.predict(x_test[:1])

In [ ]:
decomon_model = clone(vanilla_model, method="crown")